RNN을 이용한 seq2seq (Sequnce-To-Sequence) 모형은 길이가 다른 여러가지 순서열을 입력하여 마찬가지로 여러가지 길이를 가지는 순서열을 출력하는 예측 모형이다. seq2seq 모형의 여러가지 응용 분야 중 하나는 문장을 입력하였을 때 다른 언어로 된 문장으로 변환해 주는 언어 번역(language translation)이다.

## 언어 번역 모형의 구조

언어 번역 모형은 다음과 같은 인코더(Encoder)-디코더(Decoder) 구조를 사용한다. 인코더와 디코더로는 각각 RNN으로 구현된 신경망 모형이다. 각 모형의 역할을 다음과 같다.

* 인코더(Encoder) : A 언어 문장 토큰열  𝑥_1, 𝑥_2, ⋯, 𝑥_𝑇  을 그 문장의 의미/스타일 등의 요약 정보를 담고 있는 수치 벡터  𝑐 로 변환한다.
* 디코더(Decoder) : 문장의 요약정보를 담고 있는 수치 벡터  𝑐 를 B 언어 문장 토큰열  𝑦_1, 𝑦_2, ⋯, 𝑦_𝑇′ 로 변환한다.

https://datascienceschool.net/upfiles/926aa891f3a744baae57c8cdcb868ff3.png

## 인코더

인코더는 RNN으로 만들어지며 보통 LSTM이나 GRU를 사용한다.

인코더는 입력 문장 토큰열  𝑥_1, 𝑥_2, ⋯, 𝑥_𝑇 으로부터 hidden layer vector  ℎ 를 생성한다.

$$h_t = f(h_{t-1}, x_t)$$

위 식에서  𝑓 는 비선형 액티베이션 함수를 뜻한다.

인코더는 입력 언어의 확률적 언어 모형을 익히게 된다.

$$p(x_1, x_2, \cdots, x_T) = \prod_{t=1}^T p(x_t | x_{t−1}, . . . , x_1)$$

문장의 요약정보 수치벡터  𝑐 는 이렇게 생성된 hidden layer vector 열에서 만들어진다.

$$c = q(h_1, h_2, \cdots, h_T)$$

위 식에서  𝑞 는  𝑐 를 생성하는 비선형 함수이다. 가장 간단한 형태로는 다음과 같이 가장 마지막에 만들어진 모든 정보를 담고 있는 hidden layer vector를 사용하는 방법이 있을 수 있다.

$$q = h_T$$

## Bidirectional 인코더

실제로는 입력 문장의 토큰 순서를 뒤집어서 만든 reverse hidden layer vector를 사용하거나 또는 forward hiddend layser vector와 reverse hidden layer vector를 augment 시킨 벡터를 사용하면 성능이 향상되는 것으로 나타났다.

## 디코더

**디코더는 요약정보 수치벡터  𝑐 에서 실제 문장을 만들어내는 생성 모형**이다. 인코더와 마찬가지로 RNN 모형을 사용한다. 디코더는 **문장의 요약정보 벡터  𝑐 와 마지막으로 출력된 단어  𝑦_𝑡−1 로부터 다음 단어  𝑦_𝑡 를 예측**해야 한다. 즉 다음과 같이 출력 언어의 확률적 언어 모형을 담고 있어야 한다.

$$p(y_1, y_2, \cdots, y_{T'} | c) = \prod_{t=1}^{T'} p(y_t | y_{t−1}, . . . , y_1, c)$$

이 확률은 다음과 같이 디코더 hidden layer vector  𝑠  로부터 계산된다.

$$s_t = f(s_{t-1}, y_t, c)$$

$$p(y_{T'} | y_1, y_2, \cdots, c) = g(s_1, s_2, \cdots, s_t, y_{t-1}, c)$$

이렇게 계산된 토큰 확률을 softmax를 통과시켜 최대 확률을 가진 토큰을 선택한다.

## Zero-Padding and Bucketting

이론상으로는 이 신경망 모형에 입출력되는 순서열의 크기에 제한이 없지만 실제 구현시에는 크기가 고정되어야 한다.

이를 위해 **bucketting과 zero-padding 방법**을 사용한다.

* zero-padding 은 입력 토큰의 실제 크기가 미리 정해진 크기보다 작을 때 모자라는 부분을 0으로 채우는 방법이다.

* bucketting은 미리 여러가지 크기를 가진 모형을 복수개를 만들어 사용하는 방법이다. 예를 들어 10개의 토큰 순서열을 받아 10개의 토큰 순서열을 출력하는 모형을 (10, 10)-모형이라고 하면 (10, 10)-모형, (20, 20)-모형을 만들고 만약 입력 토큰열의 크기가 8이면 (10, 10)-모형을 사용하고 12이면 (20, 20)-모형을 사용하는 방법이다.

## Google Neural Machine Translation

구글의 신경망 번역기도 위에서 말한 구조를 따른다. RNN의 기본 뉴런으로는 LSTM을 사용하고 인코더, 디코더 모두 8개의 레이어를 가진다. 인코더의 경우 두번째 레이어는 토큰 순서가 뒤집힌 bidirectional 레이어를 사용한다.

https://datascienceschool.net/upfiles/cebc376b59bc4f49bcc067662f252379.png

## Attention

구글 번역 모형의 경우에는 인코더와 디코더 사이에 attention 모형이 삽입된다.

Attention이란 입력 순서열의 특정한 부분에 대해 더 집중할 수 있도록 훈련된 신경망이다.

$$s_t = \text{AttentionFunction}(y_{t'−1}, x_t)$$

$$p_t = \dfrac{\exp(s_t)/X}{\sum_{t'=1}^{T'} \exp(s_t)}$$

$$a_{t'} = \sum p_t x_t$$

## BLEU

BLEU(Bilingual Evaluation Understudy)는 번역 품질을 측정하기 위한 정량적 지수 중의 하나이다. 번역된 문장(hypothesis)와 사람이 정한 정답 문장들(reference pool)사이에 공통된 단어가 얼마나 존재하는가로 번역 품질을 측정한다.

원래는 0부터 1사이의 값을 가지는 숫자이지만 %로 표시하기도 한다. 구글의 GNMT의 경우 WMT14 Engligh-to-French 데이터에 대해 38.95 BLEU를 기록하였다.

파이썬에서는 nltk 패키지를 사용하여 BLEU를 계산할 수 있다.

In [1]:
from nltk.translate.bleu_score import sentence_bleu

In [2]:
hypothesis = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
             'ensures', 'that', 'the', 'military', 'always',
             'obeys', 'the', 'commands', 'of', 'the', 'party']
reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
              'ensures', 'that', 'the', 'military', 'will', 'forever',
              'heed', 'Party', 'commands']
reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
              'guarantees', 'the', 'military', 'forces', 'always',
              'being', 'under', 'the', 'command', 'of', 'the',
              'Party']
reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
              'army', 'always', 'to', 'heed', 'the', 'directions',
              'of', 'the', 'party']

In [3]:
sentence_bleu([reference1, reference2, reference3], hypothesis)

0.5045666840058485

## 참고 문헌
* https://www.tensorflow.org/tutorials/seq2seq
* https://arxiv.org/pdf/1609.08144.pdf